In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("EmotionCSV.csv")

In [2]:
#print(type(df))
#print(df.head())
df.drop(["Unnamed: 0"],axis = 1, inplace = True)
#print("----")

In [3]:
#print(df.head())
print(df.keys())
keyString = df.keys()
keyString += "Input"
    
df.columns = keyString
print(df.keys())

Index(['tweetid', 'datetime', 'date', 'time', 'user', 'text', 'language',
       'edited', 'textforttb', 'lematized', 'Analysis', 'word_count', 'Wut',
       'Vorfreude', 'Ekel', 'Furcht', 'Freude', 'Traurigkeit', 'Ueberraschung',
       'Vertrauen', 'neg_emotions', 'pos_emotions', 'total_neg_emotions',
       'total_pos_emotions'],
      dtype='object')
Index(['tweetidInput', 'datetimeInput', 'dateInput', 'timeInput', 'userInput',
       'textInput', 'languageInput', 'editedInput', 'textforttbInput',
       'lematizedInput', 'AnalysisInput', 'word_countInput', 'WutInput',
       'VorfreudeInput', 'EkelInput', 'FurchtInput', 'FreudeInput',
       'TraurigkeitInput', 'UeberraschungInput', 'VertrauenInput',
       'neg_emotionsInput', 'pos_emotionsInput', 'total_neg_emotionsInput',
       'total_pos_emotionsInput'],
      dtype='object')


In [4]:
df.keys()

Index(['tweetidInput', 'datetimeInput', 'dateInput', 'timeInput', 'userInput',
       'textInput', 'languageInput', 'editedInput', 'textforttbInput',
       'lematizedInput', 'AnalysisInput', 'word_countInput', 'WutInput',
       'VorfreudeInput', 'EkelInput', 'FurchtInput', 'FreudeInput',
       'TraurigkeitInput', 'UeberraschungInput', 'VertrauenInput',
       'neg_emotionsInput', 'pos_emotionsInput', 'total_neg_emotionsInput',
       'total_pos_emotionsInput'],
      dtype='object')

In [6]:
listOfTweets = df["editedInput"].tolist()
docs = np.array(listOfTweets)

"""
#Getting amount of individual words
longListofWords = " ".join(docs).split()
#print(longListofWords[0])
unique = set(longListofWords)
countOfUniqueWords = len(unique)
print(countOfUniqueWords)"""

'\n#Getting amount of individual words\nlongListofWords = " ".join(docs).split()\n#print(longListofWords[0])\nunique = set(longListofWords)\ncountOfUniqueWords = len(unique)\nprint(countOfUniqueWords)'

In [7]:
# https://medium.com/swlh/tweet-sentiment-analysis-using-python-for-complete-beginners-4aeb4456040
from sklearn.feature_extraction.text import CountVectorizer
#dir(CountVectorizer)

In [8]:
cv=CountVectorizer(binary = True)
#help(cv.fit_transform)
oneHotCv = cv.fit_transform(docs)
#print(type(oneHotCv))
#print(dir(oneHotCv))
#print(cv.get_feature_names())
oneHotArray = oneHotCv.toarray()
oneHotArray.shape

(4012, 12454)

In [9]:
"""
numberZero = 0
numberOne = 1
print("Length of Tweet {} according to textforttb: ".format(numberZero)+str(len(df["edited"][numberZero].split()))) 
print("Length of Tweet {} according to oneHotArray: ".format(numberZero)+str(sum(oneHotArray[numberZero])))
print(max(oneHotArray[0]))
print(df["edited"][numberZero].split())
print(oneHotArray[numberZero])
# All Unique Words are represented in the oneHotArray. Only "Und" ist removed, which results in a length -1.
"""

'\nnumberZero = 0\nnumberOne = 1\nprint("Length of Tweet {} according to textforttb: ".format(numberZero)+str(len(df["edited"][numberZero].split()))) \nprint("Length of Tweet {} according to oneHotArray: ".format(numberZero)+str(sum(oneHotArray[numberZero])))\nprint(max(oneHotArray[0]))\nprint(df["edited"][numberZero].split())\nprint(oneHotArray[numberZero])\n# All Unique Words are represented in the oneHotArray. Only "Und" ist removed, which results in a length -1.\n'

In [10]:
oneHotDf = pd.DataFrame(oneHotArray)
oneHotDf.columns = cv.get_feature_names_out()
oneHotDf.head()
print(oneHotDf.info())
#print(oneHotDf["text"])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4012 entries, 0 to 4011
Columns: 12454 entries, 000 to zypries
dtypes: int64(12454)
memory usage: 381.2 MB
None


In [11]:
longDF = pd.concat([df,oneHotDf], axis = 1)
#print(type(longDF))


In [12]:
#print(df.info())
print(longDF.info())
print(str(longDF.count()))
#dir(longDF)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4012 entries, 0 to 4011
Columns: 12478 entries, tweetidInput to zypries
dtypes: bool(2), int64(12467), object(9)
memory usage: 381.9+ MB
None
tweetidInput        4012
datetimeInput       4012
dateInput           4012
timeInput           4012
userInput           4012
                    ... 
zwischenschritt     4012
zwischenzeit        4012
zwischenzeitlich    4012
zynisch             4012
zypries             4012
Length: 12478, dtype: int64


In [14]:
#print(longDF.iloc[0]["edited"])
print(longDF.iloc[0][longDF.iloc[0]["editedInput"].split()])

saarlaenderinnen    1
saarlaender         1
klar                1
wechsel             1
spitze              1
landes              1
entschieden         1
sicher              1
gestalten           1
du                  1
herzlichen          1
glueckwunsch        1
ueberzeugenden      1
wahlsieg            1
ltwsaar22           1
saarland            1
Name: 0, dtype: object


In [15]:
print(longDF.keys()[:27])
print(longDF.keys()[:24])
#Shows that keys[0:25] are tweet data, thats not part of the NLP

Index(['tweetidInput', 'datetimeInput', 'dateInput', 'timeInput', 'userInput',
       'textInput', 'languageInput', 'editedInput', 'textforttbInput',
       'lematizedInput', 'AnalysisInput', 'word_countInput', 'WutInput',
       'VorfreudeInput', 'EkelInput', 'FurchtInput', 'FreudeInput',
       'TraurigkeitInput', 'UeberraschungInput', 'VertrauenInput',
       'neg_emotionsInput', 'pos_emotionsInput', 'total_neg_emotionsInput',
       'total_pos_emotionsInput', '000', '0230', '0800016'],
      dtype='object')
Index(['tweetidInput', 'datetimeInput', 'dateInput', 'timeInput', 'userInput',
       'textInput', 'languageInput', 'editedInput', 'textforttbInput',
       'lematizedInput', 'AnalysisInput', 'word_countInput', 'WutInput',
       'VorfreudeInput', 'EkelInput', 'FurchtInput', 'FreudeInput',
       'TraurigkeitInput', 'UeberraschungInput', 'VertrauenInput',
       'neg_emotionsInput', 'pos_emotionsInput', 'total_neg_emotionsInput',
       'total_pos_emotionsInput'],
      dtype='o

In [17]:
tweet_features = longDF.keys()[24:]
"""print(df.keys())
print(str(longDF["text"].head())+"\n")
print(longDF["time"].head())
print(longDF["date"].head())
print(longDF.keys()[5:].tolist().index("time")+1)
#tweet_features = longDF.keys()[10620:10630]
print(longDF[longDF.columns[10620:10630]])"""
X_trainingData = longDF[tweet_features]


In [19]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X_trainingData, y_emotionWut, random_state = 0)
emotion_Model = DecisionTreeRegressor(random_state = 1)



DecisionTreeRegressor(random_state=1)

In [27]:
from sklearn.metrics import mean_absolute_error
val_predictions = emotion_Model.predict(val_X)
print(emotion_Model.predict(val_X.head()))
print(mean_absolute_error(val_y,val_predictions))

[0. 0. 0. 0. 0.]
0.0


In [44]:
keys = list(longDF.keys()[12:20])
#print(list((keys)))
for emotion in longDF[keys]:
    print(emotion[:-5] + ": ")
    #print(longDF[emotion].head())
    y_emotion = longDF[emotion]
    train_X, val_X, train_y, val_y = train_test_split(X_trainingData, y_emotion, random_state = 0)
    emotion_Model = DecisionTreeRegressor(random_state = 1)
    emotion_Model.fit(X_trainingData,y_emotionWut)
    #print(emotion_Model.predict(val_X.head()))
    print("Mean Absolute error for {}: ".format(emotion[:-5]) + str(round(mean_absolute_error(val_y,val_predictions)*100,2))+"%\n")
    #y_emotionGoal = longDF[emotion]
    

Wut: 
Mean Absolute error for Wut: 0.0%

Vorfreude: 
Mean Absolute error for Vorfreude: 22.33%

Ekel: 
Mean Absolute error for Ekel: 7.38%

Furcht: 
Mean Absolute error for Furcht: 7.88%

Freude: 
Mean Absolute error for Freude: 27.42%

Traurigkeit: 
Mean Absolute error for Traurigkeit: 9.17%

Ueberraschung: 
Mean Absolute error for Ueberraschung: 16.45%

Vertrauen: 
Mean Absolute error for Vertrauen: 42.57%

